# FORECAST в Google Colab через GitHub (Новости → Тикеры → Влияние на свечи)

Инструкции:
- Создайте репозиторий на GitHub и добавьте туда папку проекта `news_analize` (скрипты .py и .csv).
- При приватном репозитории используйте GitHub Personal Access Token (fine-grained, только read).
- В Colab включите GPU: Runtime → Change runtime type → Hardware accelerator → GPU.
- Запускайте ячейки сверху вниз.


In [ ]:
# Параметры репозитория
REPO_URL = "https://github.com/<USER_OR_ORG>/<REPO>.git"  # замените на ваш URL
USE_TOKEN = False  # выставьте True, если репозиторий приватный
GITHUB_TOKEN = ""  # вставьте токен при приватном доступе

import os
if USE_TOKEN and GITHUB_TOKEN:
    REPO_URL_AUTH = REPO_URL.replace('https://', f'https://{GITHUB_TOKEN}@')
else:
    REPO_URL_AUTH = REPO_URL
print('Repo URL:', REPO_URL_AUTH)


In [ ]:
!git clone $REPO_URL_AUTH
!ls -la


In [ ]:
# Переходим в папку проекта (укажите правильный путь, если репозиторий иначе устроен)
import os, glob
repo_root = [d for d in os.listdir('.') if os.path.isdir(d) and d != 'sample_data']
print('Candidates:', repo_root)
%cd {repo_root[0]}/news_analize
!ls -la


In [ ]:
pip install -q -r requirements.txt
# PyTorch обычно уже есть в Colab. При необходимости можно доустановить CUDA-колёсами.


In [ ]:
# Авторазметка тикеров (если нет столбца tickers)
!python auto_label_tickers.py --news test_news.csv --out task_1_news_autolabeled.csv

import pandas as pd
x = pd.read_csv('task_1_news_autolabeled.csv')
print(x[['publish_date','title','tickers']].head())


In [ ]:
# Обучение модели новости→тикеры (при необходимости уменьшите ресурсы)
!python train_news_ticker.py --news task_1_news_autolabeled.csv --artifacts artifacts/ --epochs 3 --batch_size 64 --max_len 256

import os
print('Artifacts:', os.listdir('artifacts'))


In [ ]:
# Инференс влияния на свечи
!python infer_news_to_candles.py \
  --news task_1_news_autolabeled.csv \
  --candles public_test_candles.csv \
  --artifacts artifacts/ \
  --out nn_features.parquet

import pandas as pd
fe = pd.read_parquet('nn_features.parquet')
print(fe.head())
print('rows:', len(fe))
